In [5]:
##匯入文本
import os
import openpyxl
import re

files = os.listdir()

wb = openpyxl.Workbook()
##新建一個sheet
ws= wb.active
ws.title = "text"
ws.cell(1, 1, 'content')

i = 2
for f in files:
    if f.endswith(".txt"):
        txtname = f
        ##讀取 txt 檔案：防止讀取錯誤，讀取時需要指定編碼
        fopen = open(txtname, 'r',encoding='utf-8')
        lines = fopen.readlines()
        # 刪除空格
        if ' ' in lines:
            lines.remove(' ')
        #刪除換行符號
        if '\n' in lines:
            lines.remove('\n')
        #寫入 excel表
        for line in lines:
            line.strip()
            if '\n' in line:
                new_line=line.replace('\n', '').replace('\r', '')
                ws.cell(i, 1, new_line)
            else:
                ws.cell(i, 1, line)
        i+=1    
wb.save('corpus.xlsx')

In [54]:
##斷詞
import pandas as pd
df=pd.read_excel('corpus.xlsx')
import nltk
# nltk.download("punkt")
import spacy


## 載入spacy自然語言處理模型
# 需要再安裝python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
## spacy提供的停用字表
spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
punctuation_lst = list(r'\,\"\'\.\?\!\:\;\~\@\#\$\%\^\&\*\(\)\_\-\+\=\|\\\[\]\{\}\/\—\’')


content_token=[]
for sent in df['content']:
    ## 將review送進去處理，與此同時，記得將所有字母變成小寫，否則spacy不會將帶有大寫的字進行詞還原
    doc = nlp(sent.lower())
    
    ## 如果詞還原後的詞不在停用字典和各標點符號中，才留下。在spacy中，token有很多屬性，.lemma_是其中一種，會返回詞還原後的詞
    clean_sent = [token.lemma_ for token in doc if token.lemma_ not in spacy_stopwords+punctuation_lst]
    
    ## 使用/當作斷詞符號，並將list轉成string儲存
    content_token.append("/".join(clean_sent))

df.insert(1, column="content_tokenized", value=content_token)
df.to_excel('corpus_tokenized.xlsx', index=False)


## Frequency

In [59]:
df_token=pd.read_excel('corpus_tokenized.xlsx')

#用frequency製作的單字表
freq=[]

n=0
for a in df_token['content_tokenized']:
    b=a.split('/')
    i=nltk.FreqDist(b)
    frequ=[]
    for k in i:
        outcome=k, i[k]
        frequ.append(outcome)
    freq.insert(n,frequ)
    
    n+=1

wb = openpyxl.Workbook()

n=1
for v in freq:
    voc=[]
    fre=[]
    data=[]
    for x, z in v:
        voc.append(x)
        fre.append(z)
    data.append(voc)
    data.append(fre)
    sheet=wb.create_sheet(str(n))
    # Use the append method to add the data to the worksheet
    for row in zip(*data):
        sheet.cell(1, 1, 'Vocabulary')
        sheet.cell(1, 2, 'Frequency')
        sheet.append(row)

    # sheet.append(voc)
    # sheet.append(fre)
    n+=1
    
sheet = wb['Sheet']
wb.remove(sheet)
wb.save('Frequency.xlsx')

## Text Rank

In [ ]:
%%time
import opencc
from textrank4zh import TextRank4Keyword, TextRank4Sentence

## 因為textrank4zh模型是以簡體字訓練的，為了提高準確度，要做簡繁體轉換
cc_ts = opencc.OpenCC('tw2s')
cc_st = opencc.OpenCC('s2tw')

## 將沒有處理過的內容簡介轉成簡體後，全部合成一個string，textrank會處理
text = "".join([cc_ts.convert(intro) for intro in df_ee["Introduction"]])

## 初始化模型並分析文本
tr4w = TextRank4Keyword()
tr4w.analyze(text=text)

## 記得將關鍵詞轉成繁體，如果你習慣看繁體的話
print('關鍵詞：')
for item in tr4w.get_keywords(20, word_min_len=1):
    print(cc_st.convert(item.word), item.weight)

print()
print('關鍵片語：')
for phrase in tr4w.get_keyphrases(keywords_num=20, min_occur_num=5):
    print(cc_st.convert(phrase))

## Spacy

In [56]:
#用spacy製作的單字表
import spacy

nlp = spacy.load("en_core_web_sm")
words = set(nlp.vocab.strings)

spac=[]
n=0
for word in df_token['content_tokenized']:
    token=word.split('/')
    spa=[]
    for w in token:
        if w in words:
            spa.append(w)
    spac.insert(n,spa)
    n+=1

wb = openpyxl.Workbook()

n=1
for v in spac:
    voc=[]
    for x in v:
        voc.append(x)
    sheet=wb.create_sheet(str(n))
    # Use the append method to add the data to the worksheet
    sheet.cell(1, 1, 'Vocabulary')
    for item in voc:
        sheet.append([item])
    n+=1
    
sheet = wb['Sheet']
wb.remove(sheet)
wb.save('Spacy.xlsx')

## TF-IDF

In [57]:
from tqdm.notebook import tqdm
# 需要再安裝pip install -U scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

## 將每個內容簡介的斷詞string轉成以空格為分隔符號的string，再將所有內容簡介的斷詞用list裝起來
dataset = [" ".join(intro.split("/")) for intro in df["content_tokenized"]]

## 初始化tfidf模型，ngram_range設定為unigram到trigram，斷詞模式以'\\b\\w+\\b'斷
tfidf_vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1,3), token_pattern='\\b\\w+\\b')
## 將內容簡介的斷詞list轉換成tfidf矩陣(matrix)
tfidf = tfidf_vectorizer.fit_transform(dataset)

## 將關鍵字擷取結果及關鍵字的權重轉成表格，再以權重值由高到低排序
df_tfidf = pd.DataFrame(tfidf[0].T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["TF-IDF"])
df_tfidf = df_tfidf.sort_values('TF-IDF', ascending=False)
print(df_tfidf)

## 將tfidf訓練結果轉成可以用的特徵值。一個關鍵字為一欄位，紀錄每個內容簡介跟關鍵字之間的相似度當作特徵值
df_tfidf_f = pd.DataFrame(tfidf.toarray().transpose(), index=tfidf_vectorizer.get_feature_names()).T

 
## 有很多種方式可以篩選，例如可以降維、使用統計方法(anova)等等，這裡用比較簡單的計算平均數(小於0.009，隨機設的，沒一定)
## 紀錄哪些欄位的平均值過低
columns_todrop = [c for c in tqdm(df_tfidf_f.columns) if np.mean(df_tfidf_f[c]) < 0.009]

## 刪除那些欄位
df_tfidf_f = df_tfidf_f.drop(columns_todrop, axis=1)

                           TF-IDF
jodie                    0.553010
mouse                    0.438594
room                     0.185884
bed                      0.118146
i                        0.103394
...                           ...
glue bag flat            0.000000
glue bag                 0.000000
glue                     0.000000
glower wife know         0.000000
zooming spiral abruptly  0.000000

[23187 rows x 1 columns]


/Users/huangtsaiyuan/opt/anaconda3/envs/111corpus/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/huangtsaiyuan/opt/anaconda3/envs/111corpus/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


  0%|          | 0/23187 [00:00<?, ?it/s]

In [62]:
#將每篇的單字放在excel

df_token.insert(2, column="frequency", value=freq)
df_token.insert(3, column="spacy", value=spac)
df_final=pd.concat([df_token, df_tfidf_f],axis=1)
df_final.to_excel('voclist.xlsx', index=False)
